# Extracting symbols from image using `OpenCV`

In [34]:
import cv2
import numpy as np
from functools import cmp_to_key

In [46]:
image= cv2.imread('tests/testMath4.png')
original_image= image

gray= cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
filtered = cv2.medianBlur(gray, 5)
cv2.imshow('Image', filtered)
cv2.waitKey(0)

#imgThresholded = cv2.adaptiveThreshold(filtered, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                          #cv2.THRESH_BINARY, 199, 5)
#cv2.imshow('Image', imgThresholded)
#cv2.waitKey(0)

edges = cv2.Canny(filtered, 50,180)
cv2.imshow('Image', edges)
cv2.waitKey(0)

kernel = np.ones((5,5), np.uint8)
img_dilation = cv2.dilate(edges, kernel, iterations=5)
cv2.imshow('Image', img_dilation)
cv2.waitKey(0)


contours, hierarchy= cv2.findContours(img_dilation.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

#sorted_contours= sorted(contours, key=cv2.contourArea, reverse= True)


boundingBoxes = []
for (i,c) in enumerate(contours):
    x,y,w,h = cv2.boundingRect(c)
    boundingBoxes.append((x,y,w,h))
    """
    cropped_contour= original_image[y:y+h, x:x+w]
    image_name= "output_shape_number_" + str(i+1) + ".jpg"
    cv2.imwrite(image_name, cropped_contour)
    readimage= cv2.imread(image_name)
    cv2.imshow('Image', readimage)
    cv2.waitKey(0)
    """

rows, _, _ = image.shape
def leftRightTopBottom(tup1, tup2):
    x1, y1, _, _ = tup1
    x2, y2, _, _ = tup2
    yRegion1, yRegion2 = -1, -1
    for i in range(8):
        if y1 < rows/8 + rows*(i/8):
            yRegion1 = i
            break
    else:
        if yRegion1 == -1:
            yRegion1 = 8
    for i in range(8):
        if y2 < rows/8 + rows*(i/8):
            yRegion2 = i
            break
    else:
        if yRegion2 == -1:
            yRegion2 = 8
    
    if yRegion1 < yRegion2:
        return -1
    elif yRegion2 < yRegion1:
        return 1
    elif x1 <= x2:
        return -1
    else:
        return 1

key_leftRightTopBottom = cmp_to_key(leftRightTopBottom)
boundingBoxes = sorted(boundingBoxes, key=key_leftRightTopBottom)
#boundingBoxes = sorted(boundingBoxes, key = lambda x : x[0] + x[1])

for box in boundingBoxes:
    x,y,w,h = box
    cropped_contour= original_image[y:y+h, x:x+w]
    image_name= "output_shape_number_" + str(i+1) + ".jpg"
    cv2.imwrite(image_name, cropped_contour)
    readimage= cv2.imread(image_name)
    cv2.imshow('Image', readimage)
    cv2.waitKey(0)

cv2.destroyAllWindows()

# Reading the kaggle [dataset](https://www.kaggle.com/datasets/aidapearson/ocr-data)